# Global Daily SST Analysis: Identifying Marine Extremes with `MarEx-Detect`

In [1]:
import xarray as xr
import numpy as np
import dask
import intake
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [2]:
test_data_path = "/home/b/b382615/opt/marEx/tests/data/sst_unstructured.zarr"
ds = xr.open_zarr(str(test_data_path), chunks={}).persist()

In [3]:
sst_data = ds.to
sst_data

<xarray.DataArray 'to' (time: 14246, ncells: 1000)> Size: 57MB
dask.array<open_dataset-to, shape=(14246, 1000), dtype=float32, chunksize=(32, 1000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 114kB 1991-01-01T23:59:59 ... 2030-01-01T2...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [4]:
dask_chunks = {'time': 25}
dimensions = {
            'time': 'time',
            'xdim': 'ncells'  # Note: no 'ydim' indicates unstructured grid
        }
ncells = sst_data.sizes.get('ncells', sst_data.sizes.get('cell', 1000))
mock_neighbours = xr.DataArray(
            np.random.randint(0, ncells, (3, ncells)),
            dims=['nv', 'ncells']
        )
mock_cell_areas = xr.DataArray(
            np.ones(ncells) * 1000.0,  # Mock cell areas in m²
            dims=['ncells']
        )

In [5]:
extremes_ds = marEx.preprocess_data(
            sst_data,
            method_anomaly='shifting_baseline',
            method_extreme='hobday_extreme',
            threshold_percentile=95,
            window_year_baseline=5,  # Reduced for test data
            smooth_days_baseline=11,  # Reduced for test data
            window_days_hobday=5,  # Reduced for test data
            dimensions=dimensions,
            dask_chunks=dask_chunks,
            neighbours=mock_neighbours,
            cell_areas=mock_cell_areas
        )

In [6]:
extremes_ds

<xarray.Dataset> Size: 73MB
Dimensions:         (time: 14246, ncells: 1000, dayofyear: 366, nv: 3)
Coordinates:
  * time            (time) datetime64[ns] 114kB 1991-01-01T23:59:59 ... 2030-...
  * dayofyear       (dayofyear) int64 3kB 1 2 3 4 5 6 ... 362 363 364 365 366
  * nv              (nv) int64 24B 0 1 2
Dimensions without coordinates: ncells
Data variables:
    dat_detrend     (time, ncells) float32 57MB dask.array<chunksize=(25, 1000), meta=np.ndarray>
    mask            (ncells) bool 1kB dask.array<chunksize=(1000,), meta=np.ndarray>
    extreme_events  (time, ncells) bool 14MB dask.array<chunksize=(25, 1000), meta=np.ndarray>
    thresholds      (ncells, dayofyear) float32 1MB 0.542 0.542 ... 0.152 0.062
    neighbours      (nv, ncells) int32 12kB dask.array<chunksize=(3, 1000), meta=np.ndarray>
    cell_areas      (ncells) float32 4kB dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes:
    method_anomaly:        shifting_baseline
    method_extreme:        hobday_extreme
    threshold_percentile:  95
    preprocessing_steps:   ['Rolling climatology using 5 years', 'Smoothed wi...
    window_year_baseline:  5
    smooth_days_baseline:  11
    window_days_hobday:    5

In [7]:
extremes_ds = extremes_ds.persist()